In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import sys
from pathlib import Path
import anndata as ad
import scanpy as sc
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import seaborn as sns
from matplotlib import pyplot as plt

sys.path.insert(0, "../../scripts/methods/")
from my_paste2 import paste2_align, paste2_align_ref
sys.path.insert(0, "/home/ylu/project")
from utils import *
import time

%load_ext autoreload
%autoreload 2

2024-07-17 05:13:02.355298: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.3/lib64
2024-07-17 05:13:03.066703: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.3/lib64
2024-07-17 05:13:03.066794: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.3/lib64
2024-07-17 05:13:03.066802: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT li

In [1]:
data_folder = "../../data/STARMapPlus/"
method = 'PASTE2'
i = 3
adata = ad.read_h5ad(os.path.join(data_folder, f"sagittal{i}.h5ad"))
scale = np.linalg.norm(adata.obsm['spatial'].max(0) - adata.obsm['spatial'].min(0))
slices = split_slice(
    adata=adata,
    spatial_key='spatial_raw',
    split_num=2,
    axis=2,
)

In [4]:
align_slices[1].obsm[key_added]

array([[22652.        , 60661.70103093],
       [16183.25      , 14389.98076923],
       [11629.66326531,  7280.5       ],
       ...,
       [20116.25      , 37176.05555556],
       [12186.97391304, 16052.3826087 ],
       [ 7668.36842105, 13599.8245614 ]])

In [6]:
align_slices[1]

AnnData object with n_obs × n_vars = 5000 × 1022
    obs: 'Sub_molecular_cell_type', 'Main_molecular_tissue_region', 'Sub_molecular_tissue_region', 'Molecular_spatial_cell_type', 'slice'
    uns: 'models_align'
    obsm: 'spatial', 'spatial_raw', 'align_spatial'

In [9]:
align_slices[1].obsm[key_added]

array([[22652.        , 60661.70103093],
       [16183.25      , 14389.98076923],
       [11629.66326531,  7280.5       ],
       ...,
       [20116.25      , 37176.05555556],
       [12186.97391304, 16052.3826087 ],
       [ 7668.36842105, 13599.8245614 ]])

In [10]:
align_slices[1].obsm['spatial']

array([[22652.        , 60661.70103093],
       [16183.25      , 14389.98076923],
       [11629.66326531,  7280.5       ],
       ...,
       [20116.25      , 37176.05555556],
       [12186.97391304, 16052.3826087 ],
       [ 7668.36842105, 13599.8245614 ]])

In [ ]:
overlay_ratio_set = np.arange(0.5, 1.05, 0.05)
from tqdm import tqdm
maes = []
for overlay_ratio in tqdm(overlay_ratio_set):
    slice1 = slices[0].copy()
    slice2 = slices[1].copy()
    slice1_crop, slice2_crop = crop_slices(
        slice1, 
        slice2, 
        spatial_key='spatial', 
        overlay_ratio = overlay_ratio,
    )
    slice1, slice2 = slice1_crop.copy(), slice2_crop.copy()
    sampling_num = 5000
    sampline_idx1 = np.random.choice(slice1.shape[0], sampling_num, replace=False) if slice1.shape[0] > sampling_num else np.arange(slice1.shape[0])
    sampline_idx2 = np.random.choice(slice2.shape[0], sampling_num, replace=False) if slice2.shape[0] > sampling_num else np.arange(slice2.shape[0])
    slice1 = slice1[sampline_idx1,:]
    slice2 = slice2[sampline_idx2,:]
    spatial_key = 'spatial'
    key_added = 'align_spatial'
    
    align_slices, pis = paste2_align(
        models=[slice1, slice2],
        spatial_key=spatial_key,
        key_added=key_added,
        verbose=True,
    )
    spatial_align = align_slices[1].obsm[key_added]
    spatial_GT = align_slices[1].obsm['spatial']
    mae, mae_var = MAE(spatial_align, spatial_GT) / scale
    maes.append(mae)
np.save(f'./results/{method}_different_overlap.npy', {'maes': maes, 'overlay_ratio':overlay_ratio}, allow_pickle=True)

  0%|                                                                                                                                                                                                                                 | 0/11 [00:00<?, ?it/s]/home/ylu/project/Benchmark_alignment_methods/notebooks/ParametersStudies/../../scripts/methods/my_paste2.py:113: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]
/home/ylu/project/Benchmark_alignment_methods/notebooks/ParametersStudies/../../scripts/methods/my_paste2.py:113: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


PASTE2 model selection procedure.
Running PASTE2 with s = 0.99...
Running PASTE2 with s = 0.95...
Running PASTE2 with s = 0.9...
Running PASTE2 with s = 0.85...
Running PASTE2 with s = 0.8...
Running PASTE2 with s = 0.75...
Running PASTE2 with s = 0.7...
Running PASTE2 with s = 0.65...
Running PASTE2 with s = 0.6...
Running PASTE2 with s = 0.55...
Running PASTE2 with s = 0.5...
Running PASTE2 with s = 0.45...
Running PASTE2 with s = 0.4...
Running PASTE2 with s = 0.35...
Running PASTE2 with s = 0.3...
Running PASTE2 with s = 0.25...
Running PASTE2 with s = 0.2...
Running PASTE2 with s = 0.15...
Running PASTE2 with s = 0.1...
Running PASTE2 with s = 0.05...
Estimation of overlap percentage is 0.7
PASTE2 starts...
It.  |Loss        |Relative loss|Absolute loss
------------------------------------------------
    0|4.223698e+00|0.000000e+00|0.000000e+00
    1|1.746390e+00|1.418531e+00|2.477309e+00
    2|1.706991e+00|2.308052e-02|3.939825e-02
    3|1.696556e+00|6.150653e-03|1.043493e-02
  

/home/ylu/anaconda3/envs/GPSA/lib/python3.9/site-packages/ot/lp/__init__.py:354: UserWarning: numItermax reached before optimality. Try to increase numItermax.
  result_code_string = check_result(result_code)
  9%|███████████████████▍                                                                                                                                                                                                  | 1/11 [07:45<1:17:36, 465.62s/it]/home/ylu/project/Benchmark_alignment_methods/notebooks/ParametersStudies/../../scripts/methods/my_paste2.py:113: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]
/home/ylu/project/Benchmark_alignment_methods/notebooks/ParametersStudies/../../scripts/methods/my_paste2.py:113: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


PASTE2 model selection procedure.
Running PASTE2 with s = 0.99...
Running PASTE2 with s = 0.95...
Running PASTE2 with s = 0.9...
Running PASTE2 with s = 0.85...
Running PASTE2 with s = 0.8...
Running PASTE2 with s = 0.75...
Running PASTE2 with s = 0.7...
Running PASTE2 with s = 0.65...
Running PASTE2 with s = 0.6...
Running PASTE2 with s = 0.55...
Running PASTE2 with s = 0.5...
Running PASTE2 with s = 0.45...
Running PASTE2 with s = 0.4...
Running PASTE2 with s = 0.35...
Running PASTE2 with s = 0.3...
Running PASTE2 with s = 0.25...
Running PASTE2 with s = 0.2...
Running PASTE2 with s = 0.15...
Running PASTE2 with s = 0.1...
Running PASTE2 with s = 0.05...
Estimation of overlap percentage is 0.5
PASTE2 starts...
It.  |Loss        |Relative loss|Absolute loss
------------------------------------------------
    0|4.266617e+00|0.000000e+00|0.000000e+00
    1|1.146454e+00|2.721578e+00|3.120163e+00
    2|1.113056e+00|3.000511e-02|3.339738e-02
    3|1.107940e+00|4.617551e-03|5.115972e-03
  

/home/ylu/anaconda3/envs/GPSA/lib/python3.9/site-packages/ot/lp/__init__.py:354: UserWarning: numItermax reached before optimality. Try to increase numItermax.
  result_code_string = check_result(result_code)
 18%|██████████████████████████████████████▉                                                                                                                                                                               | 2/11 [24:36<1:57:55, 786.16s/it]/home/ylu/project/Benchmark_alignment_methods/notebooks/ParametersStudies/../../scripts/methods/my_paste2.py:113: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]
/home/ylu/project/Benchmark_alignment_methods/notebooks/ParametersStudies/../../scripts/methods/my_paste2.py:113: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


...

PASTE2 model selection procedure.
Running PASTE2 with s = 0.99...
Running PASTE2 with s = 0.95...
Running PASTE2 with s = 0.9...
Running PASTE2 with s = 0.85...
Running PASTE2 with s = 0.8...
Running PASTE2 with s = 0.75...
Running PASTE2 with s = 0.7...
Running PASTE2 with s = 0.65...
Running PASTE2 with s = 0.6...
Running PASTE2 with s = 0.55...
Running PASTE2 with s = 0.5...
Running PASTE2 with s = 0.45...
Running PASTE2 with s = 0.4...
Running PASTE2 with s = 0.35...
Running PASTE2 with s = 0.3...
Running PASTE2 with s = 0.25...
Running PASTE2 with s = 0.2...
Running PASTE2 with s = 0.15...
Running PASTE2 with s = 0.1...
Running PASTE2 with s = 0.05...
Estimation of overlap percentage is 0.5
PASTE2 starts...
It.  |Loss        |Relative loss|Absolute loss
------------------------------------------------
    0|4.260942e+00|0.000000e+00|0.000000e+00
    1|1.083671e+00|2.931953e+00|3.177272e+00


/home/ylu/anaconda3/envs/GPSA/lib/python3.9/site-packages/ot/lp/__init__.py:354: UserWarning: numItermax reached before optimality. Try to increase numItermax.
  result_code_string = check_result(result_code)
 82%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                       | 9/11 [53:31<07:03, 211.90s/it]/home/ylu/project/Benchmark_alignment_methods/notebooks/ParametersStudies/../../scripts/methods/my_paste2.py:113: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]
/home/ylu/project/Benchmark_alignment_methods/notebooks/ParametersStudies/../../scripts/methods/my_paste2.py:113: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


PASTE2 model selection procedure.
Running PASTE2 with s = 0.99...
Running PASTE2 with s = 0.95...
Running PASTE2 with s = 0.9...
Running PASTE2 with s = 0.85...
Running PASTE2 with s = 0.8...
Running PASTE2 with s = 0.75...
Running PASTE2 with s = 0.7...
Running PASTE2 with s = 0.65...
Running PASTE2 with s = 0.6...
Running PASTE2 with s = 0.55...
Running PASTE2 with s = 0.5...
Running PASTE2 with s = 0.45...
Running PASTE2 with s = 0.4...
Running PASTE2 with s = 0.35...
Running PASTE2 with s = 0.3...
Running PASTE2 with s = 0.25...
Running PASTE2 with s = 0.2...
Running PASTE2 with s = 0.15...
Running PASTE2 with s = 0.1...
Running PASTE2 with s = 0.05...
Estimation of overlap percentage is 0.7
PASTE2 starts...
It.  |Loss        |Relative loss|Absolute loss
------------------------------------------------
    0|4.284662e+00|0.000000e+00|0.000000e+00
    1|1.628671e+00|1.630773e+00|2.655991e+00
    2|1.502928e+00|8.366493e-02|1.257424e-01
    3|1.493748e+00|6.145926e-03|9.180464e-03
  

/home/ylu/anaconda3/envs/GPSA/lib/python3.9/site-packages/ot/lp/__init__.py:354: UserWarning: numItermax reached before optimality. Try to increase numItermax.
  result_code_string = check_result(result_code)
 91%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                   | 10/11 [59:38<04:19, 259.92s/it]/home/ylu/project/Benchmark_alignment_methods/notebooks/ParametersStudies/../../scripts/methods/my_paste2.py:113: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]
/home/ylu/project/Benchmark_alignment_methods/notebooks/ParametersStudies/../../scripts/methods/my_paste2.py:113: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


PASTE2 model selection procedure.
Running PASTE2 with s = 0.99...
Running PASTE2 with s = 0.95...
Running PASTE2 with s = 0.9...
Running PASTE2 with s = 0.85...
Running PASTE2 with s = 0.8...
Running PASTE2 with s = 0.75...
Running PASTE2 with s = 0.7...
Running PASTE2 with s = 0.65...
Running PASTE2 with s = 0.6...
Running PASTE2 with s = 0.55...
Running PASTE2 with s = 0.5...
Running PASTE2 with s = 0.45...
Running PASTE2 with s = 0.4...
Running PASTE2 with s = 0.35...
